In [3]:
import sys
print(sys.version)

import pandas as pd
import numpy as np
import os
import scanpy as sc
import scvelo as scv
import scirpy as ir
import matplotlib.pyplot as plt
from matplotlib import rcParams
import scanpy.external as sce
import seaborn as sns
import matplotlib.ticker as mtick

plt.rcParams['font.sans-serif']=['Arial']
sc.settings.verbosity = 3  
sc.settings.set_figure_params(dpi=120,fontsize=8, frameon=False, figsize=(3, 3), facecolor='white')


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


## Pipeline
1. preprocessing for each sample
2. combined all samples
3. normalization
4. batch effect correction
5. clustering
6. canonical markers certification

### Preprocessing & normalization function

In [ ]:
def preprocessing(adata, filter_cells, barc = '-1'):
    sc.pp.filter_genes(adata, min_cells=0.001*len(adata.obs))
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    adata.var['rb'] = adata.var_names.str.contains('^RP[SL]')
    adata.var['hb'] = adata.var_names.str.contains('^HB[APS]')
    adata.var['hsp'] = adata.var_names.str.contains('^HSP')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','rb','hb','hsp'], percent_top=None, log1p=False, inplace=True)
    adata = adata[adata.obs.n_genes_by_counts < 8000, :]
    adata = adata[adata.obs.n_genes_by_counts > 400, :]
    adata = adata[adata.obs.total_counts > 500, :]
    adata = adata[adata.obs.total_counts < 50000, :]
    adata = adata[adata.obs.pct_counts_mt < 10, :]
    adata = adata[adata.obs.pct_counts_hb < 5, :]
    if len(filter_cells)>0:
        pre_cell = len(adata.obs_names)
        cell_names = adata.obs_names.str.replace('-1',barc)
        adata = adata[~cell_names.isin(filter_cells)]
        after_cell = len(adata.obs_names)
        print("{0} cells filtered by double and mix".format(pre_cell-after_cell) )
    return adata

def normalization(adata, n_top = 5000):
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=n_top)
    #adata = adata[:, adata.var.highly_variable]
    sc.pp.regress_out(adata, ['total_counts', 'n_genes_by_counts', 
                              'pct_counts_mt', 'pct_counts_hsp'])
    sc.tl.pca(adata, svd_solver='arpack')
    return adata

def getTypeCells(adata, meta, cell_type):
    cells = list(set(meta[meta['leiden_name'].isin(cell_type)].index) & set(adata.obs.index))
    res = adata[adata.obs.index.isin(cells)]
    return res

## All cells

In [ ]:
folder = '/share2/pub/zhangyr/zhangyr/cooperation/SJTU/ESCC/cellranger_v6/'

In [ ]:
A_pre_ori = sc.read_10x_mtx(os.path.join(folder,'A_1_0/outs/filtered_feature_bc_matrix/'),cache=True)
A_post_ori = sc.read_10x_mtx(os.path.join(folder,'A_1_1/outs/filtered_feature_bc_matrix'),cache=True)
B_pre_ori = sc.read_10x_mtx(os.path.join(folder,'B_1_0/outs/filtered_feature_bc_matrix'),cache=True)
B_post_ori = sc.read_10x_mtx(os.path.join(folder,'B_1_1/outs/filtered_feature_bc_matrix'),cache=True)
C_pre_ori = sc.read_10x_mtx(os.path.join(folder,'C_1_0/outs/filtered_feature_bc_matrix'),cache=True)
C_post_ori = sc.read_10x_mtx(os.path.join(folder,'C_1_1/outs/filtered_feature_bc_matrix'),cache=True)
D_pre_ori = sc.read_10x_mtx(os.path.join(folder,'D_1_0/outs/filtered_feature_bc_matrix'),cache=True)
D_post_ori = sc.read_10x_mtx(os.path.join(folder,'D_1_1/outs/filtered_feature_bc_matrix'),cache=True)
I_pre_ori = sc.read_10x_mtx(os.path.join(folder,'I_1_0/outs/filtered_feature_bc_matrix'),cache=True)
I_post_ori = sc.read_10x_mtx(os.path.join(folder,'I_1_1/outs/filtered_feature_bc_matrix'),cache=True)
R_pre_ori = sc.read_10x_mtx(os.path.join(folder,'R_1_0/outs/filtered_feature_bc_matrix'),cache=True)
R_post_ori = sc.read_10x_mtx(os.path.join(folder,'R_1_1/outs/filtered_feature_bc_matrix'),cache=True)
N_pre_ori = sc.read_10x_mtx(os.path.join(folder,'N_1_0/outs/filtered_feature_bc_matrix'),cache=True)
N_post_ori = sc.read_10x_mtx(os.path.join(folder,'N_1_1/outs/filtered_feature_bc_matrix'),cache=True)

In [ ]:
A_pre = preprocessing(A_pre_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-1')
A_post = preprocessing(A_post_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-2')
B_pre = preprocessing(B_pre_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-3')
B_post = preprocessing(B_post_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-4')
C_pre = preprocessing(C_pre_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-5')
C_post = preprocessing(C_post_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-6')
D_pre = preprocessing(D_pre_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-7')
D_post = preprocessing(D_post_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-8')
I_pre = preprocessing(I_pre_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-9')
I_post = preprocessing(I_post_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-10')
R_pre = preprocessing(R_pre_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-11')
R_post = preprocessing(R_post_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-12')
N_pre = preprocessing(N_pre_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-13')
N_post = preprocessing(N_post_ori,list(doublet.iloc[:,0])+list(mix['Barcode']), '-14')

In [ ]:
A_con = A_pre.concatenate(A_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
B_con = B_pre.concatenate(B_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
C_con = C_pre.concatenate(C_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
D_con = D_pre.concatenate(D_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
I_con = I_pre.concatenate(I_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
R_con = R_pre.concatenate(R_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
N_con = N_pre.concatenate(N_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')

In [ ]:
adata_con = A_con.concatenate([B_con,C_con, D_con, I_con, R_con, N_con], 
                              batch_categories=['A', 'B','C','D','I','R','N'],
                              batch_key = 'patient')
adata_con.obs['treat_patient'] = adata_con.obs['treat'].str.cat(adata_con.obs['patient'],sep='_')

In [ ]:
adata_con = normalization(adata_con,5000)
sc.external.pp.bbknn(adata_con, batch_key='treat_patient',neighbors_within_batch=3)
sc.tl.umap(adata_con)
sc.tl.leiden(adata_con,resolution=0.3)
sc.pl.umap(adata_con, color=['patient', 'treat', 'leiden'])

In [ ]:
immu_markers = ["CD3D","CD2","CD3E","CD3G",   ## T cell
                "CD79A",'MS4A1',"CD79B","JCHAIN",  ## B cell
                "DCN","COL1A1","COL1A2","FN1",  ## fibroblast
                "CCL14","VWF","PECAM1","ENG",    ## endothelial cell
                "RGS5","MCAM","ACTA2","MYH11",    ## pericyte
                "LYZ","CD14","GCA","HPGDS",  ## myeloid cell
                'KRT5','SFN','KRT14','EPCAM'  ## epithelial cell
               ]  
sc.pl.umap(adata_con, color=immu_markers)

## T cells

In [ ]:
celltype = ['T cells']
A_pre_ori = getTypeCells(A_pre_ori, meta_data, celltype)
A_post_ori = getTypeCells(A_post_ori, meta_data, celltype)
B_pre_ori = getTypeCells(B_pre_ori, meta_data, celltype)
B_post_ori = getTypeCells(B_post_ori, meta_data, celltype)
C_pre_ori = getTypeCells(C_pre_ori, meta_data, celltype)
C_post_ori = getTypeCells(C_post_ori, meta_data, celltype)
D_pre_ori = getTypeCells(D_pre_ori, meta_data, celltype)
D_post_ori = getTypeCells(D_post_ori, meta_data, celltype)
I_pre_ori = getTypeCells(I_pre_ori, meta_data, celltype)
I_post_ori = getTypeCells(I_post_ori, meta_data, celltype)
R_pre_ori = getTypeCells(R_pre_ori, meta_data, celltype)
R_post_ori = getTypeCells(R_post_ori, meta_data, celltype)
N_pre_ori = getTypeCells(N_pre_ori, meta_data, celltype)
N_post_ori = getTypeCells(N_post_ori, meta_data, celltype)

In [ ]:
A_pre = preprocessing(A_pre_ori,[])
A_post = preprocessing(A_post_ori,[])
B_pre = preprocessing(B_pre_ori,[])
B_post = preprocessing(B_post_ori,[])
C_pre = preprocessing(C_pre_ori,[])
C_post = preprocessing(C_post_ori,[])
D_pre = preprocessing(D_pre_ori,[])
D_post = preprocessing(D_post_ori,[])
I_pre = preprocessing(I_pre_ori,[])
I_post = preprocessing(I_post_ori,[])
R_pre = preprocessing(R_pre_ori,[])
R_post = preprocessing(R_post_ori,[])
N_pre = preprocessing(N_pre_ori,[])
N_post = preprocessing(N_post_ori,[])

In [ ]:
A_con = A_pre.concatenate(A_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
B_con = B_pre.concatenate(B_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
C_con = C_pre.concatenate(C_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
D_con = D_pre.concatenate(D_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
I_con = I_pre.concatenate(I_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
R_con = R_pre.concatenate(R_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
N_con = N_pre.concatenate(N_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')

adata_con = A_con.concatenate([B_con,C_con, D_con, I_con, R_con, N_con], 
                              batch_categories=['A', 'B','C','D','I','R','N'],
                              batch_key = 'sample')
adata_con.obs['treat_sample'] = adata_con.obs['treat'].str.cat(adata_con.obs['sample'],sep='_')

In [ ]:
adata_con = normalization(adata_con, n_top = 5000)
sc.external.pp.bbknn(adata_con, batch_key='treat_sample',neighbors_within_batch=6)
sc.tl.umap(adata_con)
sc.tl.leiden(adata_con,resolution=0.9)
sc.pl.umap(adata_con, color=['sample','treat','leiden'])

In [ ]:
t_marker = ['CD8A','CD4','MKI67','NKG7',
            'CCL4L2','GZMK', 'FGFBP2','GNLY',
            'PDCD1','CXCL13','CXCL9','HAVCR2','CD69',
            'TCF7','SLAMF6','ID3','ID2','IL7R','CCR5','CXCR5','GZMB']
sc.pl.umap(adata_con, color=t_marker)

## B cells

In [ ]:
celltype = ['B cells']
A_pre_ori = getTypeCells(A_pre_ori, meta_data, celltype)
A_post_ori = getTypeCells(A_post_ori, meta_data, celltype)
B_pre_ori = getTypeCells(B_pre_ori, meta_data, celltype)
B_post_ori = getTypeCells(B_post_ori, meta_data, celltype)
C_pre_ori = getTypeCells(C_pre_ori, meta_data, celltype)
C_post_ori = getTypeCells(C_post_ori, meta_data, celltype)
D_pre_ori = getTypeCells(D_pre_ori, meta_data, celltype)
D_post_ori = getTypeCells(D_post_ori, meta_data, celltype)
I_pre_ori = getTypeCells(I_pre_ori, meta_data, celltype)
I_post_ori = getTypeCells(I_post_ori, meta_data, celltype)
R_pre_ori = getTypeCells(R_pre_ori, meta_data, celltype)
R_post_ori = getTypeCells(R_post_ori, meta_data, celltype)
N_pre_ori = getTypeCells(N_pre_ori, meta_data, celltype)
N_post_ori = getTypeCells(N_post_ori, meta_data, celltype)

In [ ]:
A_pre = preprocessing(A_pre_ori,[])
A_post = preprocessing(A_post_ori,[])
B_pre = preprocessing(B_pre_ori,[])
B_post = preprocessing(B_post_ori,[])
C_pre = preprocessing(C_pre_ori,[])
C_post = preprocessing(C_post_ori,[])
D_pre = preprocessing(D_pre_ori,[])
D_post = preprocessing(D_post_ori,[])
I_pre = preprocessing(I_pre_ori,[])
I_post = preprocessing(I_post_ori,[])
R_pre = preprocessing(R_pre_ori,[])
R_post = preprocessing(R_post_ori,[])
N_pre = preprocessing(N_pre_ori,[])
N_post = preprocessing(N_post_ori,[])

In [ ]:
A_con = A_pre.concatenate(A_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
B_con = B_pre.concatenate(B_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
C_con = C_pre.concatenate(C_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
D_con = D_pre.concatenate(D_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
I_con = I_pre.concatenate(I_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
R_con = R_pre.concatenate(R_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
N_con = N_pre.concatenate(N_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')

adata_con = A_con.concatenate([B_con,C_con, D_con, I_con, R_con, N_con], 
                              batch_categories=['A', 'B','C','D','I','R','N'],
                              batch_key = 'sample')
adata_con.obs['treat_sample'] = adata_con.obs['treat'].str.cat(adata_con.obs['sample'],sep='_')

In [ ]:
adata_con = normalization(adata_con, n_top = 3000)
sc.external.pp.bbknn(adata_con, batch_key='treat_sample')
sc.tl.umap(adata_con)
sc.tl.leiden(adata_con,resolution=0.5)
sc.pl.umap(adata_con, color=['sample', 'treat', 'leiden'])

In [ ]:
markers = ['CD19','IGHG1','MZB1','XBP1','CXCR5',
           'BACH2','IL4R','FCER2','TCL1A','IGHD',
           'IGHM','IGHA1','TNFRSF13B','CCR7',
           'RGS13','NEIL1','MEF2B','BCL6','MKI67','JCHAIN']
sc.pl.umap(adata_con, color=markers,cmap='magma')

## Myeloid cells

In [ ]:
celltype = ['Myeloid cells']
A_pre_ori = getTypeCells(A_pre_ori, meta_data, celltype)
A_post_ori = getTypeCells(A_post_ori, meta_data, celltype)
B_pre_ori = getTypeCells(B_pre_ori, meta_data, celltype)
B_post_ori = getTypeCells(B_post_ori, meta_data, celltype)
C_pre_ori = getTypeCells(C_pre_ori, meta_data, celltype)
C_post_ori = getTypeCells(C_post_ori, meta_data, celltype)
D_pre_ori = getTypeCells(D_pre_ori, meta_data, celltype)
D_post_ori = getTypeCells(D_post_ori, meta_data, celltype)
I_pre_ori = getTypeCells(I_pre_ori, meta_data, celltype)
I_post_ori = getTypeCells(I_post_ori, meta_data, celltype)
R_pre_ori = getTypeCells(R_pre_ori, meta_data, celltype)
R_post_ori = getTypeCells(R_post_ori, meta_data, celltype)
N_pre_ori = getTypeCells(N_pre_ori, meta_data, celltype)
N_post_ori = getTypeCells(N_post_ori, meta_data, celltype)

In [ ]:
A_pre = preprocessing(A_pre_ori,[])
A_post = preprocessing(A_post_ori,[])
B_pre = preprocessing(B_pre_ori,[])
B_post = preprocessing(B_post_ori,[])
C_pre = preprocessing(C_pre_ori,[])
C_post = preprocessing(C_post_ori,[])
D_pre = preprocessing(D_pre_ori,[])
D_post = preprocessing(D_post_ori,[])
I_pre = preprocessing(I_pre_ori,[])
I_post = preprocessing(I_post_ori,[])
R_pre = preprocessing(R_pre_ori,[])
R_post = preprocessing(R_post_ori,[])
N_pre = preprocessing(N_pre_ori,[])
N_post = preprocessing(N_post_ori,[])

In [ ]:
A_con = A_pre.concatenate(A_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
B_con = B_pre.concatenate(B_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
C_con = C_pre.concatenate(C_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
D_con = D_pre.concatenate(D_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
I_con = I_pre.concatenate(I_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
R_con = R_pre.concatenate(R_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')
N_con = N_pre.concatenate(N_post, batch_categories=['Pre', 'Post'],batch_key = 'treat')

adata_con = A_con.concatenate([B_con,C_con, D_con, I_con, R_con, N_con], 
                              batch_categories=['A', 'B','C','D','I','R','N'],
                              batch_key = 'sample')
adata_con.obs['treat_sample'] = adata_con.obs['treat'].str.cat(adata_con.obs['sample'],sep='_')

In [ ]:
adata_con = normalization(adata_con,min_cells=3, n_top = 3000)
sc.external.pp.bbknn(adata_con, batch_key='treat_sample')
sc.tl.umap(adata_con)
sc.tl.leiden(adata_con,resolution=0.5)
sc.pl.umap(adata_con, color=['sample', 'treat', 'leiden'])

In [ ]:
markers = {
    'Neutrophils':['MGP','S100P','FCGR3B','CMTM2'],
    'Monocytes':['VCAN','FCN1'],
    'Macrophages':['TUBA1B','IFI27'],
    'Mast cells':['TPSAB1','KIT'],
    'DCs':['CD1C']
}
sc.tl.dendrogram(adata_con,  'leiden_name')
sc.pl.dotplot(adata_con, markers, 'leiden_name',var_group_labels =None,
              standard_scale ='var', 
              cmap='Reds',figsize=(6,2),
              dendrogram=True,mean_only_expressed =False)

In [ ]:
DC = adata_con[adata_con.obs['leiden_name'].isin(['DCs']),]
sc.tl.leiden(DC,resolution=0.3)
sc.tl.umap(DC)
sc.pl.umap(DC, color=['sample', 'treat','treat_sample' ,'leiden'])

In [ ]:
dc_markers = {
    'cDC1':['CLEC9A','SNX3','C1orf54'],
    'cDC2':['FCN1','C1QC','C1QA','C1QB'],
    'pDC':['IRF7','CLEC4C'],
    'mDC':['FSCN1','LAMP3','CCR7']
}
sc.tl.dendrogram(DC,  'leiden')
sc.pl.dotplot(DC, dc_markers, 'leiden',var_group_labels =None,
              standard_scale ='var', 
              cmap='Reds',figsize=(12,3),
              dendrogram=True)

In [ ]:
Mono = adata_con[adata_con.obs['leiden_name'].isin(['Monocytes']),]
sc.tl.leiden(Mono,resolution=0.3)
sc.tl.umap(Mono)
sc.pl.umap(Mono, color=['sample', 'treat','treat_sample' ,'leiden'])

In [ ]:
Macro = adata_con[adata_con.obs['leiden_name'].isin(['Macrophages']),]
sc.tl.leiden(Macro,resolution=0.8)
sc.tl.umap(Macro)
sc.pl.umap(Macro, color=['sample', 'treat','treat_sample' ,'leiden'])